In [ ]:
"""
Purpose: To Redo the width and trheshold jump changes
to a cleaner version that can have different
thresholds for axon and dendrite
"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-05-26 19:04:10,845 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-26 19:04:10,849 - settings - Setting database.user to celiib
INFO - 2021-05-26 19:04:10,851 - settings - Setting database.password to newceliipass
INFO - 2021-05-26 19:04:10,870 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-05-26 19:04:10,872 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-26 19:04:10,890 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-05-26 19:04:11,429 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-26 19:04:11,434 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-26 19:04:11,436 - settings - Setting database.user to celiib
INFO - 2021-05-26 19:04:11,437 - settings - Setting database.password to newceliipass
INFO - 2021-05-26 19:04:11,438 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-26 19:04:11,439 - settings - Setting database.user to celiib
INFO - 2021-05-26 19:04:11,439 - settings - Setting database.password to newceliipass
INFO - 2021-05-26 19:04:11,441 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-26 19:04:11,991 - settings - Setting enable_python_native_blobs to True


In [3]:
#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-05-26 19:04:12,175 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-26 19:04:12,178 - settings - Setting database.user to celiib
INFO - 2021-05-26 19:04:12,180 - settings - Setting database.password to newceliipass
INFO - 2021-05-26 19:04:12,184 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-26 19:04:12,819 - settings - Setting enable_python_native_blobs to True


In [4]:
import validation_utils as vu
import neuron_visualizations as nviz
import neuron_utils as nru

# Downloading the Neuron

In [5]:
segment_id = 864691135462735678
split_index = 0

decomp_path = (minnie.DecompositionAxon() & dict(segment_id=segment_id,
                                  split_index = split_index,
                                 )).fetch1("decomposition")
pre_filt_neuron = du.filepath_to_neuron_obj(decomp_path)

Decompressing Neuron in minimal output mode...please wait


In [6]:
nviz.visualize_neuron(pre_filt_neuron,
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton


# Checking that the branch boundary transition skeleton extends past upstream node if need be

In [8]:
[k for k in dir(nru) if "endpoint" in k]

['classify_endpoint_error_branches_from_limb_concept_network',
 'closest_branch_endpoint_to_limb_starting_coordinate',
 'ordered_endpoints_on_branch_path',
 'shared_skeleton_endpoints_for_connected_branches']

In [9]:
neuron_obj = pre_filt_neuron

In [14]:
import neuron_searching as ns
result_limb_branch = ns.query_neuron(neuron_obj,
    functions_list=["parent_angle","upstream_skeletal_length","skeletal_length","axon_width"],
    query="(parent_angle>110) and (skeletal_length>5000) and (upstream_skeletal_length>1000) and (axon_width>105)",
        function_kwargs=dict(comparison_distance=1000),
    return_dataframe=False,
    limbs_to_process=[neuron_obj.axon_limb_name])

print(f"result_limb_branch = {result_limb_branch}")
nviz.plot_limb_branch_dict(neuron_obj,
                           result_limb_branch)

result_limb_branch = {'L0': array([293, 318])}

 Working on visualization type: mesh


In [19]:
interesting_node = 318

In [17]:
limb_obj = pre_filt_neuron[0]
nru.upstream_node(limb_obj,interesting_node)

359

In [20]:
nru.shared_skeleton_endpoints_for_connected_branches?

In [22]:
common_endpoint = nru.shared_skeleton_endpoints_for_connected_branches(limb_obj,
                                                    318,
                                                    359)

In [23]:
up_width,d_width,up_sk,d_sk = nru.branch_boundary_transition(limb_obj,
                              edge=[359,318],
                            upstream_common_endpoint=common_endpoint,
                            downstream_common_endpoint=common_endpoint,
                            error_on_no_network_connection=False,
                            comparison_distance=10000,
                            offset=0,
                            width_name="no_spine_median_mesh_center",
                            verbose=False)

# Develop the Function that will find a path back to the starting coordinate

In [100]:
for k in limb_obj.get_branch_names():
    nru.branch_path_to_start_node(limb_obj = neuron_obj[0],
    branch_idx = k,
    include_branch_idx = False,
    skeletal_length_min = 2000,
    verbose = False)

In [52]:
import error_detection as ed
ed.width_jump_from_upstream_min(limb_obj=neuron_obj[0],
branch_idx=318,
skeletal_length_min = 2000,
verbose = False)



-22.86456963566667

# Trying out the function as a query

In [63]:
import neuron_searching as ns
width_jump_limb_branch = ns.query_neuron(neuron_obj,
                functions_list=["width_jump_from_upstream_min"],
                query="width_jump_from_upstream_min>50",
                function_kwargs=dict(skeletal_length_min=4000),
                return_dataframe=False,
        limbs_to_process=["L0"])

In [64]:
nviz.plot_limb_branch_dict(neuron_obj,
                          width_jump_limb_branch)


 Working on visualization type: mesh


# A) Putting Together the Width Query

In [112]:
axon_limb_branch = ns.query_neuron_by_labels(neuron_obj,
                                                        matching_labels=["axon"])
dendrite_limb_branch = ns.query_neuron_by_labels(neuron_obj,
                                                        not_matching_labels=["axon"])
ed.width_jump_up_error_limb_branch_dict(neuron_obj,
                                     limb_branch_dict_restriction = axon_limb_branch,
                                     upstream_skeletal_length_min = 10000,
                                     branch_skeletal_length_min = 6000,
                                     upstream_skeletal_length_min_for_min= 4000,
                                    width_jump_max = 75,
                                    plot_final_width_jump = True,
                                       plot_upstream_skeletal_length_min=True)


upstream_skeletal_length_min = {'L0': array([  0,   1,   4,   5,   6,   7,   9,  11,  14,  15,  16,  17,  18,
        20,  21,  25,  26,  28,  29,  30,  31,  32,  34,  36,  37,  38,
        40,  41,  45,  47,  49,  51,  54,  56,  57,  58,  59,  62,  64,
        65,  67,  69,  70,  71,  80,  82,  84,  86,  90,  91,  94, 100,
       101, 105, 111, 113, 114, 116, 119, 123, 124, 125, 126, 129, 131,
       133, 134, 135, 136, 142, 144, 145, 146, 147, 149, 151, 153, 154,
       157, 161, 162, 163, 166, 168, 170, 171, 172, 174, 175, 176, 177,
       179, 180, 184, 185, 187, 189, 190, 191, 193, 194, 195, 196, 203,
       208, 209, 211, 212, 213, 214, 216, 217, 219, 220, 221, 223, 228,
       229, 235, 236, 237, 238, 240, 241, 242, 248, 249, 251, 252, 254,
       255, 257, 258, 264, 265, 267, 268, 269, 270, 272, 274, 284, 286,
       287, 288, 289, 290, 291, 293, 294, 296, 297, 299, 302, 303, 305,
       306, 310, 311, 312, 313, 314, 318, 321, 322, 323, 326, 327, 329,
       330, 332, 334, 335,

branch_skeletal_length_min = {'L0': array([293])}

 Working on visualization type: mesh


{'L0': array([293])}

# Develop the Width Query for the Dendrite and Axon (TO DO)

# For New Double Back Error

In [ ]:
"""
Purpose: 
To highlight the branches that double 



"""


In [24]:
nru.find_parent_child_skeleton_angle(limb_obj,
                                     318,
                                    359,
                                     comparison_distance=10000,
                                     plot_extracted_skeletons = True
                                    )

Parent (359):red, child (318):blue


117.46